In [1]:
### Machine Learning Online Class - Exercise 3: 
### Multi Class Classification and Neural Networks

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as op

### Load training data
dfX=pd.read_csv('ex3data_images.csv',sep=',',header=None)
dfy=pd.read_csv('ex3data_labels.csv',sep=',',header=None)
m, n = dfX.shape

### Setup the parameters you will use for this part of the exercise
input_layer_size  = 400 # 20x20 Input Images of Digits
num_labels = 10         # 10 labels, from 1 to 10 (mapped "0" to label 10)

In [2]:
### Vectorize logistic regression 
### Test case for logistic regression CostFunction with regularization
theta_t=np.array([[-2],[-1], [1], [2]])
X_t = np.hstack((np.ones([5,1]), np.reshape(np.arange(1,16),[5,3],order='F')/10));
y_t = np.array([[1],[0],[1],[0],[1]])
lambda_t = 3

def sigmoid(z):
    g=1/(1+np.exp(-z))
    return g

def lrCostFunction(theta,X,y,lam):
    m = y.shape[0]
    h_x=sigmoid(X.dot(theta))
    J=(1/m)*np.sum((-y*np.log(h_x))-((1-y)*np.log(1-h_x))) 
    diff=h_x-y
    grad=(1/m)*np.dot(diff.T,X)
    J=J+(lam/(2*m))*np.sum(np.power(theta[1:],2))
    grad[:,1:]=grad[:,1:]+(lam/m)*np.transpose(theta[1:])
    return J, grad

J, grad = lrCostFunction(theta_t, X_t, y_t, lambda_t)

print('\nCost: \n', J)
print('Expected cost: 2.534819\n')
print('Gradients:\n')
print(' \n', grad)
print('Expected gradients:\n')
print(' 0.146561\n -0.548558\n 0.724722\n 1.398003\n')


Cost: 
 2.534819396109744
Expected cost: 2.534819

Gradients:

 
 [[ 0.14656137 -0.54855841  0.72472227  1.39800296]]
Expected gradients:

 0.146561
 -0.548558
 0.724722
 1.398003



In [5]:
### One vs. All training
X=dfX.values
X=np.hstack((np.ones([m,1]), X))
y=dfy.values
all_theta = np.zeros([num_labels, n+1])
m = y.shape[0]

def lrCost(theta,X,y,lam):
    h_x=sigmoid(X.dot(theta))
    J=(1/m)*np.sum((-y*np.log(h_x))-((1-y)*np.log(1-h_x)))
    regterm=(lam/(2*m))*np.sum(np.power(theta[1:],2))
    J=J+regterm
    return J

def lrCostGrad(theta,X,y,lam):
    theta = theta[:,np.newaxis]
    h_x=sigmoid(X.dot(theta))
    diff=h_x-y
    grad=(1/m)*np.dot(diff.T,X)
    regterm=(lam/m)*np.transpose(theta[1:])
    grad[:,1:]=grad[:,1:]+(lam/m)*np.transpose(theta[1:])
    return grad

def oneVsAll(X,y,num_labels,lam): 
    for k in range(num_labels):
        print('training class', k+1)
        initial_theta=np.zeros((n+1)) #there is some issue with scipy where this has to be this shape
        c=(y==(k+1))*1
        res=op.minimize(lrCost, x0=initial_theta, jac=lrCostGrad, args=(X,c,lam),method='TNC')
        all_theta[k,:]=res.x
    return all_theta

lam=0.1
all_theta = oneVsAll(X, y, num_labels, lam)

training class 1
training class 2
training class 3
training class 4
training class 5
training class 6
training class 7
training class 8
training class 9
training class 10


In [6]:
z2=all_theta.dot(X.T)
a2=sigmoid(z2)
p=[np.argmax(z2,axis=0)+1]
print('\nTraining Set Accuracy: \n', np.mean((p==y.T)*1)*100)


Training Set Accuracy: 
 79.74


This is poorer accuracy than in the Octave assignment for the course but the optimization method is different also. More work can be done to improve this. 